In [ ]:
# pip install google-generativeai

In [1]:
import google.generativeai as genai

In [2]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML

In [3]:
import json
import time

import re
import ast

import os

In [4]:
from tenacity import retry, wait_random_exponential, stop_after_attempt
import asyncio

In [5]:
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_csv('../laptop_data.csv')
df.head()

,Brand,Model Name,Core,CPU Manufacturer,Clock Speed,RAM Size,Storage Type,Display Type,Display Size,Graphics Processor,Screen Resolution,OS,Laptop Weight,Special Features,Warranty,Average Battery Life,Price,Description
0,Dell,Inspiron,i5,Intel,2.4 GHz,8GB,SSD,LCD,"15.6""",Intel UHD,1920x1080,Windows 10,2.5 kg,Backlit Keyboard,1 year,6 hours,"35,000",The Dell Inspiron is a versatile laptop that c...
1,MSI,GL65,i7,Intel,2.6 GHz,16GB,HDD+SSD,IPS,"15.6""",NVIDIA GTX,1920x1080,Windows 10,2.3 kg,RGB Keyboard,2 years,4 hours,"55,000",The MSI GL65 is a high-performance laptop desi...
2,HP,EliteBook,i7,Intel,2.8 GHz,16GB,SSD,LED,"14""",Intel UHD,1920x1080,Windows 11,1.5 kg,Fingerprint Sensor,3 years,8 hours,"90,000",The HP EliteBook is a premium laptop designed ...
3,Lenovo,IdeaPad,i3,Intel,2.1 GHz,8GB,HDD,TN,"15.6""",Intel UHD,1366x768,Windows 10,2.2 kg,Dolby Audio,1 year,5 hours,"25,000",The Lenovo IdeaPad is a versatile laptop that ...
4,ASUS,ZenBook Pro,i9,Intel,3.1 GHz,64GB,SSD,OLED,"15.6""",NVIDIA RTX,3840x2160,Windows 10,1.8 kg,NanoEdge Display,2 years,7 hours,"200,000",The ASUS ZenBook Pro is a high-end laptop that...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Brand                 20 non-null     object
 1   Model Name            20 non-null     object
 2   Core                  20 non-null     object
 3   CPU Manufacturer      20 non-null     object
 4   Clock Speed           20 non-null     object
 5   RAM Size              20 non-null     object
 6   Storage Type          20 non-null     object
 7   Display Type          20 non-null     object
 8   Display Size          20 non-null     object
 9   Graphics Processor    20 non-null     object
 10  Screen Resolution     20 non-null     object
 11  OS                    20 non-null     object
 12  Laptop Weight         20 non-null     object
 13  Special Features      20 non-null     object
 14  Warranty              20 non-null     object
 15  Average Battery Life  20 non-null     obje

In [7]:
df.Description

0     The Dell Inspiron is a versatile laptop that c...
1     The MSI GL65 is a high-performance laptop desi...
2     The HP EliteBook is a premium laptop designed ...
3     The Lenovo IdeaPad is a versatile laptop that ...
4     The ASUS ZenBook Pro is a high-end laptop that...
5     The Acer Predator is a powerhouse laptop desig...
6     The Microsoft Surface Laptop is a premium devi...
7     The Lenovo ThinkPad is a powerful laptop desig...
8     The HP Pavilion is a budget-friendly laptop th...
9     The ASUS ROG Strix G is a high-performance gam...
10    The Dell XPS 15 is a premium laptop that combi...
11    The Lenovo ThinkPad X1 Carbon is a sleek and l...
12    The Acer Swift 3 is a lightweight and affordab...
13    The Apple MacBook Air is a sleek and lightweig...
14    The MSI Prestige 14 is a compact and stylish l...
15    The ASUS ZenBook 13 is a lightweight and power...
16    The Dell Precision 5550 is a high-performance ...
17    The HP ENVY x360 is a versatile 2-in-1 con

<img src="../stage_system_design.png"/>

# Stage 1: Intent Clarity and Intent Confirmation

<img src="../image+3.jpg"/>

In [8]:
GOOGLE_API_KEY = open("gemini_API_Key.txt", "r").read().strip()
genai.configure(api_key=GOOGLE_API_KEY)

## Step1: Initialise the conversation with base prompt
- Role of the model
- Guidelines for the model
- Instruction within the transcripts
- Chain of Thoughts
- Few Shot examples (samples)

- Returns a list [{"role": "model", "parts": [system_message]}]

In [9]:
def initialize_conversation():
    delimiter = "####"

    example_user_dict = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "low",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}

    example_user_req = {'GPU intensity': "_",
                        'Display quality': "_",
                        'Portability': "_",
                        'Multitasking': "_",
                        'Processing speed': "_",
                        'Budget': "_"}

    system_message = open("../Prompts/initialize_conversation.txt", "r").read().strip().format(delimiter=delimiter, 
                                                                           example_user_dict=example_user_dict, 
                                                                           example_user_req=example_user_req)
    conversation = [{"role": "model", "parts": [system_message]}]
    # conversation = system_message
    return conversation

#### Displaying the base prompt

In [ ]:
debug_conversation = initialize_conversation()
print(debug_conversation[0]['parts'][0])

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

#### Moderation on user input does not work in Gemini. 
The API is not publicly available. 
It can only provide safety ratings on generated response

In [ ]:
# Moderation does not work well with Gemini
def moderation_check(user_input):
    MODEL = 'gemini-1.5-flash'
    model = genai.GenerativeModel(MODEL)
    response = model.generate_content(user_input, safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    })

    print(response)
    # for rating in response.candidates[0].safety_ratings:
        # if rating.HarmProbability > HarmCategory.:
        #     return False

    # return True
        

In [ ]:
moderation_check("I want to kill them")

### `get_chat_completions` - Define the function get_chat_completions with two parameters
- prompt
- Is Json response type

The prompt this time will contain the user query as well
- If json response type then we will return a json
- Else, we will return text result

`@retry` - In case of failure, retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay

In [10]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gemini-1.5-flash'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""
    gemini_model = genai.GenerativeModel(MODEL)
    # Only 1 candidate_count is allowed as of now
    if json_format:
        gen_config = genai.types.GenerationConfig(candidate_count=1,
                                              temperature=0, 
                                              response_mime_type="application/json")
        chat_completion_json = gemini_model.generate_content(input, 
                                           generation_config=gen_config)

        output = json.loads(chat_completion_json.candidates[0].content.parts[0].text)
        return output
    else:
        gen_config = genai.types.GenerationConfig(candidate_count=1,
                                              temperature=0)
        chat_completion_json = gemini_model.generate_content(input, 
                                           generation_config=gen_config)

        output = chat_completion_json.candidates[0].content.parts[0].text
        return output

In [ ]:
# Testing the OpenAI functions defined above
input_prompt ='What is the capital of France?'
messages = [{'role':'user','parts':[input_prompt]}]
messages

In [ ]:
get_chat_completions(messages), get_chat_completions(messages, json_format=True)

### `iterate_llm_response` - Iterate the same prompt for `n` number of times and verify the consistency of the result

Calls a specified function repeatedly and prints the results.
This function is designed to test the consistency of a response from a given function.
It calls the function multiple times (default is 10) and prints out the iteration count,
the function's response(s).<br/>

Args:
- funct (function): The function to be tested. This function should accept a single argument
                  and return the response value(s).
- debug_response (dict): The input argument to be passed to 'funct' on each call.
- num (int, optional): The number of times 'funct' will be called. Defaults to 10.

Returns:
    This function only returns the results to the console.

In [11]:
def iterate_llm_response(funct, debug_response, num = 10):
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer'
# iterate_llm_response(get_chat_completions, messages)

## Step2: Initialise the user query in the base prompt
- Initialise user query in base prompt
- Send it to the model to evaluate
- Model will return text output or json output based on the parameter sent to `get_chat_completion` function
- Add system response to the base prompt

#### Initialise user query in base prompt

In [ ]:
debug_user_input = "Hi, I am Anand. I need a laptop for coding."

In [ ]:
debug_conversation.append({"role": "user", "parts": [debug_user_input]})
print(debug_conversation[1]["parts"][0])

In [ ]:
display(debug_conversation)

#### Send it to the model to evaluate - Text response

In [ ]:
debug_response_assistant = get_chat_completions(debug_conversation)
print(debug_response_assistant)

#### Send it to the model to evaluate - Json response

In [ ]:
debug_response_assistant_json = get_chat_completions(debug_conversation, json_format=True)
print(debug_response_assistant_json)

#### Add system response to the base prompt

In [ ]:
debug_conversation.append({"role": "model", "parts": [debug_response_assistant]})

In [ ]:
display(debug_conversation)

### intent_confirmation_layer()
This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks 

    Parameters:
    - response_assistant (str): The input text containing user requirements captured through 6 keys:
        'GPU intensity', 'Display quality', 'Portability', 'Multitasking', 'Processing speed', and 'Budget'.

    Returns:
    - str: A one-word string in JSON format indicating if the values for the specified keys are correctly filled.
        - 'Yes' if the values are correctly filled for all keys ('GPU intensity', 'Display quality', 'Portability',
          'Multitasking', 'Processing speed') based on the importance as stated by the user.
        - 'No' otherwise.

    Note:
    - The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on their importance as stated by the user.
    - The input text should be structured such that it contains the necessary keys and their corresponding values.
    - The function uses OpenAI's Chat Completion API to evaluate the correctness of the input values.

if the following properties for the user has been captured or not
   - GPU intensity
   - Display quality
   - Portability
   - Multitasking
   - Processing speed
   - Budget

In [12]:
def intent_confirmation_layer(response_assistant):
    MODEL = 'gemini-1.5-flash'
    gemini_model = genai.GenerativeModel(MODEL)
    
    delimiter = "####"
    allowed_values = {'low','medium','high'}
    
    prompt = open("../Prompts/intent_confirmation_layer.txt", "r").read().strip().format(allowed_values=allowed_values, 
                                                                                         delimiter=delimiter)
    messages=[{"role": "model", "parts":[prompt] },
              {"role": "user", "parts":[f"""Here is the input: {response_assistant}"""]}]

    gen_config = genai.types.GenerationConfig(candidate_count=1,
                                              temperature=0, 
                                              response_mime_type="application/json")
    intent_confirmation_json = gemini_model.generate_content(messages, 
                                       generation_config=gen_config)

    output = json.loads(intent_confirmation_json.candidates[0].content.parts[0].text)

    return output

In [ ]:
print(debug_response_assistant)

In [ ]:
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
display(debug_confirmation)

#### Extract the dictionary from the response

In [13]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'GPU intensity': 'high',
                'Display quality': 'high',
                'Portability': 'medium',
                'Multitasking': 'high',
                'Processing speed': 'high',
                'Budget': '200000'}

    prompt = open("../Prompts/dictionary_present.txt", "r").read().strip().format(delimiter=delimiter, 
                                                                                        user_req=user_req)
    messages = [{"role": "model", "parts":[prompt]},
                {"role": "user", "parts":[f"""Here is the user input: {response}""" ]}]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

In [ ]:
debug_response_assistant_n = f"""Thank you for providing your budget.
Based on your budget of 80,000 INR, I will consider this while recommending suitable laptop options for you.
Here is the final recommendation for your laptop:
- GPU intensity: high
- Display quality: high
- Portability: low
- Multitasking: high
- Processing speed: medium
- Budget: 80,000 INR

Please note that these specifications are based on your requirements for surfing and a decent display within your budget.
Let me know if there's anything else I can assist you with!"""

In [ ]:
iterate_llm_response(dictionary_present, debug_response_assistant_n)

In [ ]:
debug_conversation = initialize_conversation()
debug_user_input = "Hi, I am Anand. I need a laptop for coding."
# debug_moderation = moderation_check(debug_user_input)
debug_conversation.append({"role": "user", "parts": [debug_user_input]})
debug_response_assistant = get_chat_completions(debug_conversation)
# debug_moderation = moderation_check(debug_response_assistant)
debug_conversation.append({"role": "model", "parts": [debug_response_assistant]})
debug_confirmation = intent_confirmation_layer(debug_response_assistant)
# After a series of conversation...
response_dict_n = dictionary_present(debug_response_assistant_n)
print(response_dict_n)

# Stage 2: Product Extraction and Product Mapping

<img src="../Stage+2.jpg"/>

### product_map_layer()
This function is responsible for extracting key features and criteria from laptop descriptions. Here's a breakdown of how it works:
-  Use a prompt that assign it the role of a Laptop Specifications Classifier, whose objective is to extract key features and classify them based on laptop descriptions.
- Provide step-by-step instructions for extracting laptop features from description.
- Assign specific rules for each feature (e.g., GPU Intensity, Display Quality, Portability, Multitasking, Processing Speed) and associate them with the appropriate classification value (Low, Medium, or High).
- Includes Few Shot Prompting (sample conversation between the user and assistant) to demonstrate the expected result of the feature extraction and classification process.

#### Extracts key features from a laptop description and classifies them based on predefined rules.
- Args: laptop_description (str): The description of the laptop to be analyzed.
- Returns:
        - dict: A dictionary containing the classified features of the laptop.
            Keys represent different aspects such as GPU intensity, Display quality, Portability,
            Multitasking, and Processing speed, each mapped to a value indicating the classification
            (low, medium, or high) based on the extracted feature

In [14]:
def product_map_layer(laptop_description):
    delimiter = "#####"
    lap_spec = {
        "GPU intensity":"(Type of the Graphics Processor)",
        "Display quality":"(Screen Resolution)",
        "Portability":"(Laptop Weight)",
        "Multitasking":"(RAM)",
        "Processing speed":"(CPU Manufacturer, Core)"
    }
    values = {'low','medium','high'}

    prompt = open("../Prompts/product_map_layer.txt", "r").read().strip().format(delimiter=delimiter, 
                                                                                 lap_spec=lap_spec, 
                                                                                 values=values,
                                                                                 laptop_description=laptop_description)

    input = f"""Follow the prompt instructions step-by-step and output the dictionary in JSON format for the following laptop {laptop_description}."""

    messages=[{"role": "model", "parts":[prompt]},{"role": "user","parts":[input]}]

    response = get_chat_completions(messages)

    return response

In [ ]:
debug_laptop_description = f"""
The Dell Inspiron is a versatile laptop that combines powerful performance and affordability.
It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing.
With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity.
The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience.
Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage.
Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience.
With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment.
All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users.
"""
display(product_map_layer(debug_laptop_description))
print(type(product_map_layer(debug_laptop_description)))

In [ ]:
iterate_llm_response(product_map_layer, debug_laptop_description)

### Update the Laptop CSV with the Laptop Feature
- Read each row of the Laptop database and apply product_map_layer
- This will provide a dictionary, store it in laptop feature function
- Use this new column for comparing with User requirements

In [15]:
def generate_dictionary_with_wait(columnSeries, isDebug):
    updated_features = []
    for item in columnSeries:
        time.sleep(10)
        result = product_map_layer(item)
        updated_features.append(result)
        if isDebug:
            print(result)
    return updated_features
    
def update_laptop_csv_with_feature(isDebug = False):
    laptop_csv = pd.read_csv("../laptop_data.csv")
    updated_features = generate_dictionary_with_wait(laptop_csv["Description"], isDebug=isDebug)
    laptop_csv["laptop_feature"] = updated_features
    laptop_csv.to_csv("../updated_laptop_data.csv")

In [ ]:
update_laptop_csv_with_feature()

In [ ]:
updated_laptop_csv = pd.read_csv("../updated_laptop_data.csv")
updated_laptop_csv

In [16]:
def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"


    dictionary_matches = re.findall(regex_pattern, string)


    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()


        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)
    return dictionary

### compare_laptops_with_user()
This function compares the user's profile with the different laptops and come back with the top  recommendations. It will perform the following steps:
    - It will take the user requirements dictionary as input
    - Filter the laptops based on their price, keeping only the ones within the user's budget.
    - Calculate a score for each laptop based on how well it matches the user's requirements.
    - Sort the laptops based on their scores in descending order.
    - Return the top 3 laptops as a JSON-formatted string.

#### Compares laptops based on user requirements and returns the top 3 matching laptops in JSON format.

    Parameters:
    - user_req_string (str): A string representing user requirements in JSON format.

    Returns:
    - str: A JSON string containing information about the top 3 matching laptops based on user requirements.

In [17]:
def get_budget_in_int(requirements_dic):
    return requirements_dic.get('budget'.capitalize(), '0') if type(requirements_dic.get('budget'.capitalize(), '0')) == int else int(requirements_dic.get('budget'.capitalize(), '0').replace(',', '').split()[0])
    
def compare_laptops_with_user(user_req_string):
    laptop_df= pd.read_csv('../updated_laptop_data.csv')
    user_requirements = user_req_string if type(user_req_string) == dict else extract_dictionary_from_string(user_req_string)
    budget =  get_budget_in_int(user_requirements)

    filtered_laptops = laptop_df.copy()
    filtered_laptops["Price"] = filtered_laptops["Price"].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops["Price"] <= budget].copy()
    
    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0


        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key.lower(), None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1


        filtered_laptops.loc[index, 'Score'] = score


    # Sort the laptops by score in descending order and return the top 5 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
    
    return top_laptops

In [ ]:
compare_laptops_with_user(response_dict_n)

In [18]:
def product_validation(recommendation):
    # recommendation = recommendation.drop("Unnamed", axis=1)
    validated_laptops = []
    for index, row in recommendation.iterrows():
        if row.Score > 2:
            validated_laptops.append(row)

    df = pd.concat(validated_laptops, axis=1).transpose()
    return df

In [ ]:
product_validation(compare_laptops_with_user(response_dict_n))

# Stage 3: Product Recommendation

<img src="../Stage+3.jpg"/>

## Product Recommendation Layer

Finally, we come to the product recommendation layer. It takes the output from the `compare_laptops_with_user` function in the previous layer and provides the recommendations to the user. It has the following steps.
1. Initialize the conversation for recommendation.
2. Generate the recommendations and display in a presentable format.
3. Ask questions basis the recommendations.

In [22]:
def initialize_conv_reco(products):
    system_message = open("../Prompts/initialize_conv_reco.txt", "r").read().strip().format(products=products)
    conversation = [{"role": "user", "parts": [system_message] }]
    return conversation

In [ ]:
debug_recommendation_json = json.loads(product_validation(compare_laptops_with_user(response_dict_n)).to_json(orient="records"))

In [ ]:
print(list(map(lambda x: x["Description"], debug_recommendation_json)))

In [ ]:
debug_conversation_reco = initialize_conv_reco(debug_recommendation_json)
debug_conversation_reco

In [ ]:
# json_text = json.dumps(debug_recommendation_json, sort_keys=True, indent=4, separators=(',', ': '))
# json_text

In [ ]:
# debug_conversation_reco = initialize_conv_reco(json_text)
# debug_conversation_reco

In [ ]:
debug_system_top3_recommendation = get_chat_completions(debug_conversation_reco)

In [ ]:
debug_conversation_reco.append({"role":"model", "parts":[debug_system_top3_recommendation]})
debug_conversation_reco

In [ ]:
debug_conversation_reco.append({"role":"user", "parts":["Which of these is best suited for travel. Recommend only one"]})

In [ ]:
final_result = get_chat_completions(debug_conversation_reco)
print(final_result)

In [ ]:
os.path.isfile("updated_laptop_data.csv")

In [20]:
def check_moderation(user_input):
    return "Not Flagged" # moderation_check(user_input) # - Gemini does not support moderation on user input
    
def dialogue_mgmt_system(isDebug = False):

    print("Initialising system, please wait....\n")

    if os.path.isfile("../updated_laptop_data.csv") == False:
        update_laptop_csv_with_feature()

    print("System is ready \n")
    
    conversation = initialize_conversation()
    print("--"*20 + "Shop Assitant" + "--"*20)
    print("""
    Hello, I am your laptop assistant. Please feel free to ask me about laptop recommendations. \n
    You can tell me about your needs like \n
    - what is the purpose of the laptop
    - Do you need heavy processing laptop
    - Do you need it for travel purposes, etc

    Please note: I do not take inputs to talk to about any specific brand as I am suppose to be non-biased. 
    Once I find a good recommendatio for you, then you can ask me about that laptop.\n
    Let's start!!!
    """ + '\n')
    top_3_laptops = None
    user_input = ''


    while(user_input != "exit"):
        print("--"*20 + "User" + "--"*20)
        user_input = input("")

        
        if check_moderation(user_input) == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break


        if top_3_laptops is None:
            conversation.append({"role": "user", "parts": [user_input]})


            response_assistant = get_chat_completions(conversation)

            if check_moderation(user_input) == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            confirmation = intent_confirmation_layer(response_assistant)

            if check_moderation(user_input) == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break


            if "No".lower() in json.dumps(confirmation, sort_keys=True, indent=4, separators=(',', ': ')).lower():
                conversation.append({"role": "model", "parts": [response_assistant]})
                print("--"*20 + "Shop Assitant" + "--"*20)
                print(f"\n{response_assistant}\n")
                
                if isDebug:
                    print(f"\n{confirmation}\n")
            else:
                print("--"*20 + "Shop Assitant" + "--"*20)
                print(f"\n{response_assistant}\n")
                
                if isDebug:
                    print(f"\n{confirmation}\n")
                    
                response = dictionary_present(response_assistant)

                if check_moderation(user_input) == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break




                print("--"*20 + "Shop Assitant" + "--"*20)
                
                if isDebug:
                    print(f'\n{response}\n')
                    
                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptops_with_user(response)


                validated_reco = product_validation(top_3_laptops)


                if len(validated_reco) == 0:
                    print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
                    break


                conversation_reco = initialize_conv_reco(validated_reco)
                recommendation = get_chat_completions(conversation_reco)


                if check_moderation(user_input) == 'Flagged':
                    print("Sorry, this message has been flagged. Please restart your conversation.")
                    break


                conversation_reco.append({"role": "user", "parts": [f"This is my user profile {response}"]})


                conversation_reco.append({"role": "model", "parts": [recommendation]})


                print("--"*20 + "Shop Assitant" + "--"*20)
                print(f'{recommendation}\n')


        else:
            conversation_reco.append({"role": "user", "parts": [user_input]})


            response_asst_reco = get_chat_completions(conversation_reco)

            if check_moderation(user_input) == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print("--"*20 + "Shop Assitant" + "--"*20)
            print(f'\n{response_asst_reco}\n')
            conversation.append({"role": "model", "parts": [response_asst_reco]})

# Run Simulation in Debug Mode

In [23]:
dialogue_mgmt_system(isDebug=True)

Initialising system, please wait....

System is ready 

----------------------------------------Shop Assitant----------------------------------------

    Hello, I am your laptop assistant. Please feel free to ask me about laptop recommendations. 

    You can tell me about your needs like 

    - what is the purpose of the laptop
    - Do you need heavy processing laptop
    - Do you need it for travel purposes, etc

    Please note: I do not take inputs to talk to about any specific brand as I am suppose to be non-biased. 
    Once I find a good recommendatio for you, then you can ask me about that laptop.

    Let's start!!!
    

----------------------------------------User----------------------------------------


 Hi, this is Mohit Dubey and I am looking for a coding laptop


----------------------------------------Shop Assitant----------------------------------------

Hi Mohit! That's great!  Coding laptops can be quite versatile. To help me find the perfect one for you, could you tell me a bit more about your coding needs?  For example, what kind of coding do you do?  Do you work with large datasets, complex software, or primarily web development?  Knowing this will help me understand your requirements for processing power, storage, and other features. 



{'result': 'No', 'reason': 'Missing keys'}

----------------------------------------User----------------------------------------


 I will working on Machine Learning, Deep Learning and NLPs and models I will run locally


----------------------------------------Shop Assitant----------------------------------------

Okay, that's helpful!  So you'll be doing a lot of computationally intensive work with Machine Learning, Deep Learning, and NLPs.  This means you'll need a laptop with strong processing power and a dedicated GPU.  

To make sure I can find the best fit for you, could you tell me your budget for the laptop?  Knowing this will help me narrow down the options and find something that meets your needs without breaking the bank. 



{'result': 'No', 'reason': 'Budget is not provided'}

----------------------------------------User----------------------------------------


...

----------------------------------------Shop Assitant----------------------------------------

Okay, that's helpful!  Since portability and display quality aren't a priority, we can focus on finding a laptop with the best possible performance for your Machine Learning, Deep Learning, and NLP workloads. 

Based on our conversation, here's what I've gathered about your needs:

* **GPU Intensity:** High (You'll be doing a lot of demanding work with Machine Learning, Deep Learning, and NLPs)
* **Display Quality:** Medium (Not a priority)
* **Portability:** Low (Not a priority)
* **Multitasking:** High (You'll likely be running multiple programs and processes simultaneously)
* **Processing Speed:** High (You'll need a powerful processor to handle your workloads)
* **Budget:** 300000 INR

I'm confident that with this information, I can find you a great laptop that meets your needs.  Do you have any other preferences or requirements I should be aware of? 



{'result': 'Yes'}

---------------

 Which one is best suited for me. Suggest only one


----------------------------------------Shop Assitant----------------------------------------

Based on your profile, the **Dell Precision 5550** is the best suited for you. 

Here's why:

* **High GPU intensity:** The Dell Precision 5550 boasts a powerful NVIDIA Quadro graphics card, ideal for demanding tasks like video editing, 3D rendering, and gaming.
* **High Multitasking:** With a Xeon processor and 32GB RAM, this laptop can handle multiple demanding applications simultaneously without breaking a sweat.
* **High Processing Speed:** The Xeon processor ensures lightning-fast performance for even the most complex tasks.
* **Budget:** While it's the most expensive option, it falls within your budget of Rs. 300,000.

While the other laptops offer great features, the Dell Precision 5550 excels in the areas you prioritize: GPU intensity, multitasking, and processing speed. 


----------------------------------------User----------------------------------------


 exit


----------------------------------------Shop Assitant----------------------------------------

Based on your profile, the **Dell Precision 5550** is the best suited for you. 

Here's why:

* **High GPU intensity:** The Dell Precision 5550 boasts a powerful NVIDIA Quadro graphics card, ideal for demanding tasks like video editing, 3D modeling, and gaming.
* **High Multitasking:** With 32GB of RAM and a Xeon processor, this laptop can handle multiple demanding applications simultaneously without slowing down.
* **High Processing Speed:** The Xeon processor ensures lightning-fast performance for even the most complex tasks.
* **Medium Display Quality:** The 15.6" OLED display offers vibrant colors and deep blacks, meeting your medium display quality preference.
* **Low Portability:** While not the lightest, the 1.83 kg weight is manageable for occasional movement.
* **Budget:** The Dell Precision 5550 fits within your budget of Rs. 300,000.

While the ASUS ZenBook Pro also offers high per

# Run Simulation

In [24]:
dialogue_mgmt_system()

Initialising system, please wait....

System is ready 

----------------------------------------Shop Assitant----------------------------------------

    Hello, I am your laptop assistant. Please feel free to ask me about laptop recommendations. 

    You can tell me about your needs like 

    - what is the purpose of the laptop
    - Do you need heavy processing laptop
    - Do you need it for travel purposes, etc

    Please note: I do not take inputs to talk to about any specific brand as I am suppose to be non-biased. 
    Once I find a good recommendatio for you, then you can ask me about that laptop.

    Let's start!!!
    

----------------------------------------User----------------------------------------


 Hi, this is Mohit Dubey and I am looking for a coding laptop


----------------------------------------Shop Assitant----------------------------------------

Hi Mohit! That's great!  Coding laptops can be quite versatile. To help me find the perfect one for you, could you tell me a bit more about your coding needs?  For example, what kind of coding do you do?  Do you work with large datasets, complex software, or primarily web development?  Knowing this will help me understand your requirements for processing power, storage, and other features. 


----------------------------------------User----------------------------------------


 I will working on Machine Learning, Deep Learning and NLPs and models I will run locally


----------------------------------------Shop Assitant----------------------------------------

Okay, that's helpful!  So you'll be doing a lot of computationally intensive work with Machine Learning, Deep Learning, and NLPs.  This means you'll need a laptop with strong processing power and a dedicated GPU.  

To make sure I can find the best fit for you, could you tell me your budget for this laptop?  Knowing your budget will help me narrow down the options and find something that meets your needs without breaking the bank. 


----------------------------------------User----------------------------------------


...

----------------------------------------Shop Assitant----------------------------------------

{'GPU intensity': 'high', 'Display quality': 'medium', 'Portability': 'medium', 'Multitasking': 'high', 'Processing speed': 'high', 'Budget': '250000'} 


----------------------------------------Shop Assitant----------------------------------------
Thank you for providing all the information. Kindly wait, while I fetch the products: 

----------------------------------------Shop Assitant----------------------------------------
Here is the recommendation:

1. **ASUS ZenBook Pro**: i9 Intel CPU, 3.1 GHz, 64GB RAM, SSD, NVIDIA RTX Graphics, 15.6" OLED Display, 3840x2160 Resolution, Windows 10, 2 years warranty, 7 hours battery life, Rs. 200000
2. **Dell Precision 5550**: Xeon Intel CPU, 2.6 GHz, 32GB RAM, SSD, NVIDIA Quadro Graphics, 15.6" OLED Display, 3840x2160 Resolution, Ubuntu, 3 years warranty, 6 hours battery life, Rs. 250000
3. **Dell XPS 15**: i9 Intel CPU, 2.8 GHz, 32GB RAM, SSD, NVIDI

 Which one is best suited?


----------------------------------------Shop Assitant----------------------------------------

Based on your profile, the **ASUS ZenBook Pro** seems like the best fit. Here's why:

* **High GPU intensity:** The ZenBook Pro boasts an NVIDIA RTX graphics card, which is ideal for demanding tasks like gaming and video editing. 
* **High Multitasking:** With 64GB of RAM, the ZenBook Pro can handle multiple applications and programs simultaneously without any lag.
* **High Processing Speed:** The i9 Intel CPU with a 3.1 GHz clock speed ensures lightning-fast performance for all your tasks.
* **Medium Portability:** The ZenBook Pro is a 15.6" laptop, making it suitable for carrying around, although it's not the lightest option.
* **Medium Display Quality:** The OLED display offers excellent color accuracy and contrast, meeting your medium display quality requirement.
* **Budget:** The ZenBook Pro fits comfortably within your budget of Rs. 250,000.

While the Dell Precision 5550 also offers hi

 exit


----------------------------------------Shop Assitant----------------------------------------

Okay, I understand.  Let me know if you have any other questions. 


